<a href="https://colab.research.google.com/github/reesha-rsh/MLb4/blob/main/Homework/HW9_Boosting_XGBoost%2C_LightGBM%2C_CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Homework:

- use the dataset and validation approach which you used for previous homeworks
- train xgboost, lightgbm, and catboost models and find the best hyperparameters for each algorithm
- compare results between them and previously trained random forest

# Init

In [191]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import make_scorer, fbeta_score, classification_report, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import pandas as pd
import numpy as np


from google.colab import drive
drive.mount('/content/drive')

train_full = pd.read_csv("/content/drive/MyDrive/MLb4/EDA Titanic/train.csv")
test_full = pd.read_csv("/content/drive/MyDrive/MLb4/EDA Titanic/test.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [192]:
random_state = 42

In [193]:
def generate(df,age_median,fare_median):
  useless_features = ['Name','Ticket','Cabin']
  data_cleaned = df
  data_cleaned = data_cleaned.drop(columns = useless_features)

  # generate binary values using get_dummies
  data_cleaned = pd.get_dummies(data_cleaned, columns=['Sex'],prefix=["Sex"])
  data_cleaned = pd.get_dummies(data_cleaned, columns=['Embarked'],prefix=["Embarked"])

  # Check for NaN values in the DataFrame
  nan_mask = data_cleaned.isnull()
  # Count the number of NaN values in each column
  nan_count_per_column = data_cleaned.isnull().sum()

  data_cleaned['Age'] = data_cleaned['Age'].fillna(age_median)
  data_cleaned['Fare'] = data_cleaned['Fare'].fillna(fare_median)

  return data_cleaned


In [194]:
not_features = ['PassengerId', 'Survived']

In [195]:
# get medians that will fill NaNs in generate func
age_median = train_full['Age'].median()
print(age_median)
fare_median = train_full['Fare'].median()
print(fare_median)

28.0
14.4542


In [196]:
train = generate(train_full,age_median=age_median,fare_median=fare_median)
print(train)

X_train = train.drop(not_features, axis = 1)
y_train = train['Survived']

     PassengerId  Survived  Pclass   Age  SibSp  Parch     Fare  Sex_female  \
0              1         0       3  22.0      1      0   7.2500           0   
1              2         1       1  38.0      1      0  71.2833           1   
2              3         1       3  26.0      0      0   7.9250           1   
3              4         1       1  35.0      1      0  53.1000           1   
4              5         0       3  35.0      0      0   8.0500           0   
..           ...       ...     ...   ...    ...    ...      ...         ...   
886          887         0       2  27.0      0      0  13.0000           0   
887          888         1       1  19.0      0      0  30.0000           1   
888          889         0       3  28.0      1      2  23.4500           1   
889          890         1       1  26.0      0      0  30.0000           0   
890          891         0       3  32.0      0      0   7.7500           0   

     Sex_male  Embarked_C  Embarked_Q  Embarked_S  

In [197]:
test = generate(test_full,age_median=age_median,fare_median=fare_median)
X_test = test.drop('PassengerId' , axis = 1)

In [198]:
# Initialize the StratifiedKFold object
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

In [199]:
# Define the custom scoring function with the desired beta value
beta = 0.5
custom_scorer = make_scorer(fbeta_score, beta=beta)

#XGBoost

In [10]:
import xgboost as xgb

In [11]:
xgb_train = xgb.DMatrix(X_train, y_train, feature_names=X_train.columns)

num_rounds = 10000
param = {
    #default
    "objective": "binary:logistic",
    "eta": 0.01,
    "verbosity": 0,
    "nthread": 10,
    "random_seed": random_state,
    "feval": custom_scorer,


    # regularization parameters
    "max_leaves": 16,
    "subsample": 0.7,
    "colsample_bytree": 0.9,
    "gamma": 0,
    "min_child_weight": 5,


    #lightgbm approach
    "tree_method": "hist",
    "grow_policy": "lossguide"
}

results = xgb.cv(param, xgb_train, num_rounds, early_stopping_rounds=50,folds=skf, verbose_eval=100)

[0]	train-logloss:0.68906+0.00013	test-logloss:0.68935+0.00010
[100]	train-logloss:0.46626+0.00400	test-logloss:0.47945+0.01062
[200]	train-logloss:0.40395+0.00450	test-logloss:0.43058+0.01665
[300]	train-logloss:0.37409+0.00492	test-logloss:0.41286+0.02047
[400]	train-logloss:0.35697+0.00516	test-logloss:0.40710+0.02327
[500]	train-logloss:0.34452+0.00540	test-logloss:0.40435+0.02584
[600]	train-logloss:0.33471+0.00581	test-logloss:0.40283+0.02860
[700]	train-logloss:0.32643+0.00612	test-logloss:0.40101+0.02967
[800]	train-logloss:0.31920+0.00632	test-logloss:0.39998+0.03174
[900]	train-logloss:0.31266+0.00646	test-logloss:0.39878+0.03275
[1000]	train-logloss:0.30672+0.00666	test-logloss:0.39818+0.03344
[1100]	train-logloss:0.30145+0.00673	test-logloss:0.39750+0.03425
[1200]	train-logloss:0.29651+0.00681	test-logloss:0.39664+0.03440
[1245]	train-logloss:0.29443+0.00689	test-logloss:0.39678+0.03457


In [12]:
model = xgb.XGBClassifier(
    #default
    objective= "binary:logistic",
    eta= 0.01,
    verbosity= 0,
    nthread= 10,
    random_seed= random_state,
    feval= custom_scorer,


    # regularization parameters
    max_leaves= 16,
    subsample= 0.7,
    colsample_bytree= 0.9,
    gamma= 0,
    min_child_weight= 5,


    #lightgbm approach
    tree_method= "hist",
    grow_policy= "lossguide"
)

model.fit(X_train, y_train)

fbeta = cross_val_score(model, X_train, y_train, scoring=custom_scorer, cv=skf).mean()
accuracy = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=skf).mean()

print("F-beta Score (beta={}): {:.4f}".format(beta, fbeta))
print("Accuracy: {:.4f}".format(accuracy))

F-beta Score (beta=0.5): 0.7880
Accuracy: 0.8126


In [13]:
y_pred = model.predict(X_test)
submission_data = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': y_pred})
submission_data.to_csv("/content/drive/MyDrive/MLb4/EDA Titanic/submissionXgb.csv", index=False)

**SUBMISSION SCORE:** 0.77511

# LightGBM

In [15]:
import lightgbm as lgb

In [147]:
# Define the F-beta score function
def custom_fbeta_score(y_true,y_pred):
    beta = 0.5  # Set your desired beta value
    y_true_binary = (y_true > 0.5).astype(int)
    return 'f_beta', fbeta_score(y_true_binary, y_pred.get_label(), beta=beta), True

In [188]:
parameters = {
    #default
    "objective": "binary",
    "learning_rate": 0.01,
    "num_threads": 10,
     "metric": "None",
    "seed":  [random_state],

    #regularization
    "colsample_bytree": 0.7,
    "subsample": 0.7,
    "subsample_freq": 1,
    "min_data_in_leaf": 7,
    'force_col_wise': 'true'

}


n_rounds = 10000

lgb_train = lgb.Dataset(X_train, label=y_train, free_raw_data=False)
result = lgb.cv(parameters, lgb_train, n_rounds,feval=custom_fbeta_score, folds=skf, callbacks=([lgb.early_stopping(stopping_rounds=20),lgb.log_evaluation(period=10, show_stdv=True)]), eval_train_metric=True)

[LightGBM] [Info] Number of positive: 273, number of negative: 439
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 10
[LightGBM] [Info] Number of positive: 274, number of negative: 439
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 713, number of used features: 10
[LightGBM] [Info] Number of positive: 274, number of negative: 439
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 713, number of used features: 10
[LightGBM] [Info] Number of positive: 274, number of negative: 439
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 713, number of used features: 10
[LightGBM] [Info] Number of positive: 273, number of negative: 440
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 713, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pa

In [186]:
parameters = {
    #default
    "objective": "binary",
    "learning_rate": 0.05,
    "num_threads": 10,
     "metric": "None",
    "seed":  [random_state],

    #regularization
    "colsample_bytree": 0.8,
    "subsample": 0.8,
    "subsample_freq": 1,
    "min_data_in_leaf": 5,
    'force_col_wise': 'true'

}


n_rounds = 10000

lgb_train = lgb.Dataset(X_train, label=y_train, free_raw_data=False)
result = lgb.cv(parameters, lgb_train, n_rounds,feval=custom_fbeta_score, folds=skf, callbacks=([lgb.early_stopping(stopping_rounds=20),lgb.log_evaluation(period=10, show_stdv=True)]), eval_train_metric=True)

[LightGBM] [Info] Number of positive: 273, number of negative: 439
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 10
[LightGBM] [Info] Number of positive: 274, number of negative: 439
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 713, number of used features: 10
[LightGBM] [Info] Number of positive: 274, number of negative: 439
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 713, number of used features: 10
[LightGBM] [Info] Number of positive: 274, number of negative: 439
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 713, number of used features: 10
[LightGBM] [Info] Number of positive: 273, number of negative: 440
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 713, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pa

In [190]:
lgb_model = lgb.LGBMClassifier(
    objective= "binary",
    learning_rate= 0.01,
    num_threads= 10,
    metric= "None",
    seed=  random_state,
    colsample_bytree= 0.7,
    subsample= 0.9,
    subsample_freq= 1,
    min_data_in_leaf= 5,
    force_col_wise= True)


lgb_model.fit(X_train, y_train,eval_metric =custom_fbeta_score)

fbeta = cross_val_score(lgb_model, X_train, y_train, scoring=custom_scorer, cv=skf).mean()
accuracy = cross_val_score(lgb_model, X_train, y_train, scoring='accuracy', cv=skf).mean()

print("F-beta Score (beta={}): {:.4f}".format(beta, fbeta))
print("Accuracy: {:.4f}".format(accuracy))

y_pred = lgb_model.predict(X_test)
submission_data = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': y_pred})
submission_data.to_csv("/content/drive/MyDrive/MLb4/EDA Titanic/submissionLgb.csv", index=False)


[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Info] Number of positive: 342, number of negative: 549
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 891, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383838 -> initscore=-0.473288
[LightGBM] [Info] Start training from score -0.473288
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Info] Number of positive: 273, number of negative: 439
[LightGBM] [Info] Total Bins 202
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 10
[Lig

**SUBMISSION SCORE:** 0.77272

# CatBoost

In [17]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 8.4 MB/s eta 0:00:00


In [18]:
categorical_features_indices = np.where(X_train.dtypes != float)[0]

In [53]:
class FBetaMetric:

    def __init__(self, beta):
        self.beta = beta

    @staticmethod
    def get_fbeta(y_true, y_pred, beta):
        y_pred_binary = (y_pred >= 0.5).astype(int)
        return fbeta_score(y_true, y_pred_binary, beta=beta)

    def is_max_optimal(self):
        return True  # greater is better

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        y_true = np.array(target).astype(int)
        approx = np.array(approxes[0])
        score = self.get_fbeta(y_true, approx, self.beta)
        return score, 1

    def get_final_error(self, error, weight):
        return error

custom_fbeta_metric = FBetaMetric(beta=beta)



In [60]:
import catboost as ctb
parameters = {
    "loss_function": "Logloss",
    "eval_metric": custom_fbeta_metric,
    "iterations": 10000,
    "learning_rate": 0.01,
    "random_seed": random_state,
    "od_wait": 20,
    "od_type": "Iter",
    "thread_count": 10,
    "subsample": 0.7,
    "colsample_bylevel": 0.9
}

ctb_data = ctb.Pool(X_train,y_train,cat_features=categorical_features_indices)
result = ctb.cv(ctb_data, parameters, folds=skf, seed=random_state, verbose_eval=20, plot=False)

/usr/local/lib/python3.10/dist-packages/catboost/core.py:6655: UserWarning: Can't optimze method "evaluate" because self argument is used
  return _cv(params, pool, fold_count, inverted, partition_random_seed, shuffle, stratified,


Training on fold [0/5]
0:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 57.3ms	remaining: 9m 33s
20:	learn: 0.6097561	test: 0.6369427	best: 0.6369427 (20)	total: 264ms	remaining: 2m 5s
40:	learn: 0.7911001	test: 0.7746479	best: 0.7746479 (25)	total: 440ms	remaining: 1m 46s

bestTest = 0.7746478873
bestIteration = 25

Training on fold [1/5]
0:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 10.8ms	remaining: 1m 48s
20:	learn: 0.6484642	test: 0.6250000	best: 0.6250000 (20)	total: 266ms	remaining: 2m 6s
40:	learn: 0.7957245	test: 0.7500000	best: 0.7670455 (25)	total: 504ms	remaining: 2m 2s

bestTest = 0.7670454545
bestIteration = 25

Training on fold [2/5]
0:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 11.6ms	remaining: 1m 56s
20:	learn: 0.7227139	test: 0.6845238	best: 0.6845238 (20)	total: 278ms	remaining: 2m 12s
40:	learn: 0.7951807	test: 0.7552083	best: 0.7552083 (23)	total: 650ms	remaining: 2m 37s

bestTest = 0.7552083333
bestIteration = 23

T

In [79]:
clf = ctb.CatBoostClassifier(
    loss_function = "Logloss",
    eval_metric = custom_fbeta_metric,
    # iterations= 10000,
    learning_rate= 0.01,
    random_seed= random_state,
    od_wait = 20,
    od_type= "Iter",
    thread_count= 10,
    subsample= 0.7,
    colsample_bylevel= 0.9)


clf.fit(X_train, y_train,cat_features=categorical_features_indices)

/usr/local/lib/python3.10/dist-packages/catboost/core.py:2268: UserWarning: Can't optimze method "evaluate" because self argument is used
  _check_train_params(params)


0:	learn: 0.0000000	total: 102ms	remaining: 1m 41s
1:	learn: 0.0000000	total: 116ms	remaining: 58.1s
2:	learn: 0.0000000	total: 132ms	remaining: 43.7s
3:	learn: 0.0000000	total: 139ms	remaining: 34.6s
4:	learn: 0.0000000	total: 147ms	remaining: 29.3s
5:	learn: 0.0000000	total: 157ms	remaining: 26.1s
6:	learn: 0.0000000	total: 165ms	remaining: 23.4s
7:	learn: 0.0000000	total: 175ms	remaining: 21.7s
8:	learn: 0.0000000	total: 189ms	remaining: 20.8s
9:	learn: 0.0000000	total: 196ms	remaining: 19.4s
10:	learn: 0.0000000	total: 204ms	remaining: 18.3s
11:	learn: 0.0000000	total: 210ms	remaining: 17.3s
12:	learn: 0.0000000	total: 228ms	remaining: 17.3s
13:	learn: 0.0000000	total: 237ms	remaining: 16.7s
14:	learn: 0.0000000	total: 245ms	remaining: 16.1s
15:	learn: 0.3246753	total: 251ms	remaining: 15.4s
16:	learn: 0.5846154	total: 258ms	remaining: 14.9s
17:	learn: 0.6268012	total: 266ms	remaining: 14.5s
18:	learn: 0.6473829	total: 273ms	remaining: 14.1s
19:	learn: 0.6758530	total: 280ms	remain

In [83]:
fbeta = cross_val_score(clf, X_train, y_train, scoring=custom_scorer, cv=skf).mean()
accuracy = cross_val_score(clf, X_train, y_train, scoring='accuracy', cv=skf).mean()

print("F-beta Score (beta={}): {:.4f}".format(beta, fbeta))
print("Accuracy: {:.4f}".format(accuracy))

Streaming output truncated to the last 5000 lines.
2:	total: 5.02ms	remaining: 1.67s
3:	total: 6.33ms	remaining: 1.57s
4:	total: 7.69ms	remaining: 1.53s
5:	total: 9.02ms	remaining: 1.49s
6:	total: 10.3ms	remaining: 1.46s
7:	total: 11.3ms	remaining: 1.4s
8:	total: 12.9ms	remaining: 1.42s
9:	total: 14.6ms	remaining: 1.44s
10:	total: 16.2ms	remaining: 1.45s
11:	total: 17.4ms	remaining: 1.43s
12:	total: 18.8ms	remaining: 1.42s
13:	total: 20.1ms	remaining: 1.42s
14:	total: 21.5ms	remaining: 1.41s
15:	total: 22.9ms	remaining: 1.41s
16:	total: 24.4ms	remaining: 1.41s
17:	total: 26.6ms	remaining: 1.45s
18:	total: 28.9ms	remaining: 1.49s
19:	total: 30.9ms	remaining: 1.51s
20:	total: 32.5ms	remaining: 1.51s
21:	total: 34.5ms	remaining: 1.53s
22:	total: 36.5ms	remaining: 1.55s
23:	total: 37.8ms	remaining: 1.54s
24:	total: 39.2ms	remaining: 1.53s
25:	total: 40.8ms	remaining: 1.53s
26:	total: 42.3ms	remaining: 1.53s
27:	total: 43.7ms	remaining: 1.52s
28:	total: 44.6ms	remaining: 1.49s
29:	total: 45

In [84]:
y_pred = clf.predict(X_test)
submission_data = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': y_pred})
submission_data.to_csv("/content/drive/MyDrive/MLb4/EDA Titanic/submissionCat.csv", index=False)

**SUBMISSION SCORE:** 0.78229

#Conclusion


- **Random forest**
  - Submission Accuracy: 0.76555
  - F-beta Score (beta=0.5): 0.8119
  - Accuracy: 0.8440
- **XGBoost**
  - Submission Accuracy: 0.77751
  - F-beta Score (beta=0.5): 0.7880
  - Accuracy: 0.8126
- **LightGBM**
  - Submission Accuracy: 0.77272
  - F-beta Score (beta=0.5): 0.8230
  - Accuracy: 0.8395
- **CatBoost**:
  - F-beta Score (beta=0.5): 0.8097
  - Accuracy: 0.8384
  - Accuracy: 0.78229

Generally boosting models perform better. For CatBoost I have got the highest F beta score but Accuracy on all boostings are lower than Random Forest.